In [3]:
import numpy as np
import pandas as pd
import glob
import os
import skimage
from skimage.io import imread, imsave
import warnings
warnings.filterwarnings("ignore")

In [4]:
masks = imread("../data/full_mask_final_segmentation_hwatershed_bg500_90%.tif")

In [6]:
masks.shape

(33, 1079, 1095)

In [5]:
masks = imread("../data/full_mask_final_segmentation_hwatershed_bg500_90%.tif")
masks = np.swapaxes(np.swapaxes(masks, 0, 1), 1, 2)
masks.shape

(1079, 1095, 33)

In [6]:
cell_properties = skimage.measure.regionprops_table(np.squeeze(masks), properties=['area', 'centroid'])

In [22]:
cell_properties

{'area': array([ 66.,  30.,  66., ...,  46., 161., 127.]),
 'centroid-0': array([1017.34848485,  969.36666667,  925.36363636, ...,  367.91304348,
         366.04968944,  361.85826772]),
 'centroid-1': array([468.63636364, 441.63333333, 528.5       , ..., 572.        ,
        538.67701863, 459.83464567]),
 'centroid-2': array([31.81818182, 32.        , 31.81818182, ...,  0.        ,
         0.77018634,  0.23622047])}

In [7]:
List_folder = [f.path for f in os.scandir("../data/SIMILARITY10_In115") if f.is_dir()]

N_X = 1079
N_Y = 1095
N_Z = 33
N_channels = len(List_folder)

In [15]:
List_folder

['../data/SIMILARITY10_In115/Sm147',
 '../data/SIMILARITY10_In115/Ir193',
 '../data/SIMILARITY10_In115/La139',
 '../data/SIMILARITY10_In115/Er166',
 '../data/SIMILARITY10_In115/Nd142',
 '../data/SIMILARITY10_In115/In113',
 '../data/SIMILARITY10_In115/Pr141',
 '../data/SIMILARITY10_In115/Nd146',
 '../data/SIMILARITY10_In115/Sm154',
 '../data/SIMILARITY10_In115/Nd143',
 '../data/SIMILARITY10_In115/Pt198',
 '../data/SIMILARITY10_In115/In115',
 '../data/SIMILARITY10_In115/Yb173',
 '../data/SIMILARITY10_In115/Ir191',
 '../data/SIMILARITY10_In115/Yb172',
 '../data/SIMILARITY10_In115/Nd150',
 '../data/SIMILARITY10_In115/Dy161',
 '../data/SIMILARITY10_In115/Gd158',
 '../data/SIMILARITY10_In115/Dy164',
 '../data/SIMILARITY10_In115/Yb176',
 '../data/SIMILARITY10_In115/Eu151',
 '../data/SIMILARITY10_In115/Sm149',
 '../data/SIMILARITY10_In115/Ho165',
 '../data/SIMILARITY10_In115/Er167',
 '../data/SIMILARITY10_In115/Yb174',
 '../data/SIMILARITY10_In115/Yb171',
 '../data/SIMILARITY10_In115/Lu175',
 

In [8]:
from pathlib import Path



In [9]:
Final_image = np.zeros((N_X,N_Y,N_Z,N_channels))
for k in range(len(List_folder)):
    directory = Path(List_folder[k])
    List_files = sorted([f for f in directory.iterdir() if f.is_file()])

    Temp_image = np.zeros((N_X,N_Y,N_Z))
    for i in range(len(List_files)):
        Temp_image[:,:,i] = imread(List_files[i], plugin='tifffile')
    Final_image[:,:,:,k] = Temp_image

In [10]:
Final_image.shape

(1079, 1095, 33, 36)

In [ ]:
List_metals = []

# IMPORT LIBRARIES
import numpy as np
import pandas as pd
import os
import skimage

from skimage.io import imread
from pathlib import Path

# DOWNLOADING DATA AND LIST VARIABLES
masks = imread("../data/full_mask_final_segmentation_hwatershed_bg500_90%.tif")
masks = np.swapaxes(np.swapaxes(masks, 0, 1), 1, 2) # swapping x and y axes, then z and y axes

fold_list = [f.path for f in os.scandir("../data/SIMILARITY10_In115") if f.is_dir()]
n_chan = len(fold_list) # number of channels == n of cell markers
chan_list = [] # list of channel names

x_dim, y_dim, z_dim = 1079, 1095, 33
image = np.zeros((x_dim, y_dim, z_dim, n_chan)) # with channels dim added

def reshape_image(fold_list):
    """
    Reshape the image data into a 4D numpy array (with channels added)

    @fold_list: list of folders containing the original image data
    """
    for k in range(len(fold_list)):
        directory = Path(fold_list[k])
        file_list = sorted([f for f in directory.iterdir() if f.is_file()])

        image_temp = np.zeros((x_dim, y_dim, z_dim))
        for i in range(len(file_list)):
            image_temp[:, :, i] = imread(file_list[i], plugin = 'tifffile')
        image[:, :, :, k] = image_temp
    return image

def channel_order(fold_list):
    """
    Save the order of the ion channels in a csv file
    
    @fold_list: list of folders containing the original image data
    """
    for k in range(len(fold_list)):
        chan_list.append(fold_list[k][-5:])
        chan_list = pd.DataFrame(chan_list)
        chan_list.to_csv("../data/Order_ion_channels.csv")

# CELL INTENSITIES, COORDINATES AND CHANNEL ORDER
image = reshape_image(fold_list)
cell_int = skimage.measure.regionprops_table(np.squeeze(masks), intensity_image = image,
                                                   properties = ['mean_intensity'])
cell_props = skimage.measure.regionprops_table(np.squeeze(masks), 
                                               properties = ['area', 'centroid'])
channel_order(fold_list)
pd.DataFrame(cell_props).to_csv("../data/cell_coordinates.csv", header = ['area','x',"y","z"])
pd.DataFrame(cell_int).to_csv("../data/cell_intensities.csv")
for k in range(len(List_folder)):
    List_metals.append(List_folder[k][-5:])
    

In [ ]:
list(List_metals[0])

36

In [24]:
List_metals = pd.DataFrame(List_metals)
List_metals.to_csv("Order_ion_channels.csv")

In [24]:
Final_image = imread("Reshaped_data.tiff")

In [28]:
Cell_intensities = skimage.measure.regionprops_table(masks,intensity_image=Final_image,
                                                   properties=['mean_intensity'])


In [30]:
Cell_properties = pd.DataFrame(cell_properties)
Cell_intensities = pd.DataFrame(Cell_intensities)

In [31]:
Cell_properties.to_csv("New_cell_properties.csv",header=['Area','X',"Y","Z"])
Cell_intensities.to_csv("New_cell_intensities.csv")

In [32]:
Cell_properties

,area,centroid-0,centroid-1,centroid-2
0,66.0,1017.348485,468.636364,31.818182
1,30.0,969.366667,441.633333,32.000000
2,66.0,925.363636,528.500000,31.818182
3,139.0,915.071942,566.043165,31.683453
4,39.0,912.128205,529.871795,32.000000
...,...,...,...,...
23915,40.0,368.750000,753.500000,0.000000
23916,163.0,367.478528,450.975460,0.779141
23917,46.0,367.913043,572.000000,0.000000
23918,161.0,366.049689,538.677019,0.770186
